2/10/2016

# Analysis for VR3 Assignment 2

The primary equation in this analysis is:
$$R_i^{stock} = \alpha + \beta \times R_i^{market} + \epsilon_i$$


Notes
* The min date and max dates for a year are not 1/1 and 12/31. 

References
* [Calculating Stock Beta, Volatility, and More by Gouthaman Balaraman](http://gouthamanbalaraman.com/blog/calculating-stock-beta.html)
* [What's the difference between alpha and beta? By Investopedia](http://www.investopedia.com/ask/answers/102714/whats-difference-between-alpha-and-beta.asp)

In [1]:
%matplotlib inline
import os
from datetime import date
import pandas as pd
from tabipy import Table, TableRow, TableCell, TableHeaderRow
from pandas_datareader.data import DataReader
import matplotlib.pyplot as plt

# Get Data

In [2]:
start_date = date(2010,1,29)
end_date = date.today()

#Specifiy companies here
company_syms = ['AAPL', 'BAC', 'GE', 'XOM', 'WMT']
#^GSPC is the S&P index
df_market = DataReader('^GSPC','yahoo', start_date, end_date)

results = []

df_stocks = pd.DataFrame()
for sym in company_syms:
    df_tmp = DataReader(sym, 'yahoo', start_date, end_date)
    df_tmp['company'] = sym
    df_stocks = pd.concat([df_tmp, df_stocks])

#Add year column to for groupby key.
df_stocks["year"] = df_stocks.index.year

# Compute
* Annual return on investment
    * Absolute
    * Relative to market
* Measure of associated risk (e.g. historical volatility)

In [3]:
df_total = pd.DataFrame({'stock_adjclose': df_stocks['Adj Close'],
                         'market_adjclose': df_market['Adj Close'],
                         'year': df_stocks['year'],
                         'company': df_stocks['company']}, index=df_stocks.index)

#Compute Returns
df_total['stock_return'] = df_total['stock_adjclose'] / df_total['stock_adjclose'].shift(1) - 1
df_total['market_return'] = df_total['market_adjclose'] / df_total['market_adjclose'].shift(1) - 1
df_total = df_total.dropna()

In [5]:
def compute_annual_return(g):
    """
    Computes the relative difference between stock prices on the minimum date, and the maximum. 
    """
    date_min = pd.Series(g.index).min()
    date_max = pd.Series(g.index).max()
    annual_return = (g['stock_adjclose'].loc[date_max] - g['stock_adjclose'].loc[date_min]) / \
                     g['stock_adjclose'].loc[date_min]
    return annual_return

In [6]:
annual_returns = df_total.groupby(['company', 'year']).apply(compute_annual_return)
annual_returns = pd.DataFrame(annual_returns)
annual_returns = annual_returns.reset_index()
#Take mean of annual returns over each stock.
mean_annual_returns = annual_returns.groupby('year').apply(np.mean)
mean_annual_returns = mean_annual_returns.rename_axis({0: 'mean_annual_return'}, axis=1)

In [7]:
def regress(g):
    X = g['market_return']
    Y = g['stock_return']
    if len(X) != len(Y):
        raise Exception('Regression data are not same size.')
    m = pd.stats.ols.OLS(Y, X)
    alpha, beta = m.beta['intercept'], m.beta['x']
    sd = np.std(X)
    r2 = m.r2
    
    #Annualize
    N = 365
    alpha = alpha*N
    sd = sd*np.sqrt(N)
    return pd.Series( {'alpha': alpha, 'beta': beta, 
                       'r2': r2, 'volatility': sd})

In [8]:
reg_df = df_total.groupby('year').apply(regress)
results_df = pd.concat([reg_df, mean_annual_returns['mean_annual_return']], axis=1)
results_df = results_df.reset_index()
results_df.to_csv('./data/results.csv', index=None)

### Compile Report PDF

Compile PDF of `vr3-assignment-02-v2-writeup.ipynb` with custom latex template. 

In [57]:
%%writefile latex_nocode.tplx
((*- extends 'article.tplx' -*))
% Disable input cells
((* block input_group *))
((* endblock input_group *))
((* block title *))
\title{VR3 Assignment 2}
\author{Phil Pope}
\date{\today}
((* endblock title *))

Overwriting latex_nocode.tplx


In [85]:
!jupyter nbconvert vr3-assignment-02-v2-writeup.ipynb --to=pdf --template=latex_nocode.tplx

[NbConvertApp] Converting notebook vr3-assignment-02-v2-writeup.ipynb to pdf
[NbConvertApp] Writing 15752 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running pdflatex 3 times: ['pdflatex', 'notebook.tex']
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 150986 bytes to vr3-assignment-02-v2-writeup.pdf
